In [1]:
import json
import pdfplumber
import spacy
import pandas as pd
import numpy as np
import os
import unidecode
import PyPDF2
from tika import parser
from spacy.matcher import PhraseMatcher
from spacy import displacy
from progress_bar.progress_bar import printProgressBar
import torch
import gc
import multiprocessing
import sys
from flair.data import Sentence
from flair.models import SequenceTagger

        

In [2]:
tagger = SequenceTagger.load("flair/ner-english-large")

2022-07-30 10:09:42,860 loading file /home/heider/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-07-30 10:10:26,025 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [3]:
nlp_spa = spacy.load('en_core_web_trf')

In [4]:
#spacy.prefer_gpu()

In [5]:
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_compare = spacy.load('en_core_web_lg')
#nlp_person = spacy.load('en_core_web_trf')
#nlp_person = spacy.load('es_dep_news_trf')

In [383]:
def clean_break_line(docs):
    new_docs = docs.replace('\n',"")
    return new_docs
    
def clean_spaces_text(text):
    new_text = " ".join(text.split())
    return(new_text)
    
def clean_points_text(docs):
    new_docs = docs.replace('.',"")
    return new_docs

def clean_commas_text(docs):
    new_docs = docs.replace(',',"")
    return new_docs

def clean_apostrophe(docs):
    new_docs = docs.replace("'","")
    return new_docs

def clean_accents(text):
    unaccented_string = unidecode.unidecode(text)
    return unaccented_string
    #lower_u_string = unaccented_string.lower()
    #return lower_u_string

def split_authors(text):
    return text.split(";")

def contains_text_list(names, authors):
    df_names = []
    df_authors = []
    df_text = []
    df_lower_authors = []
    for aut in authors:
        for elem in names:
            df_names.append(elem)
            df_lower_authors.append(clean_accents(aut.lower()))
            df_text.append(contains_text(clean_accents(elem.lower()), clean_accents(aut.lower())))
            df_authors.append(aut)
    df = {'names': df_names,'author': df_authors, 'author_lower': df_lower_authors,'contains': df_text}
    df = pd.DataFrame(df)
    #print(df)
    return df

def name_in_title(data_frame, title):
    can_tutor = data_frame.copy()
    #print(title)
    can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
    #print(can_tutor)
    
    false_values = can_tutor[can_tutor['is_in_title']==False]
    return false_values
   

def contains_text(text1, text2):
    text_1 = text1.split(" ").copy()
    text_1 = list(map(clean_accents, text_1))
    text_1 = list(map(clean_apostrophe, text_1))
    text_1 = list(map(clean_commas_text, text_1))
    text_1 = list(map(clean_points_text, text_1))
    text_2 = text2.split(" ").copy()
    text_2 = list(map(clean_accents, text_2))
    text_2 = list(map(clean_apostrophe, text_2))
    text_2 = list(map(clean_commas_text, text_2))
    text_2 = list(map(clean_points_text, text_2))
    #print(text_1)
    #print(text_2)
    for word_1 in text_1:
        if word_1 != None and len(word_1)>2:
            for word_2 in text_2:
                if word_2!= None and word_1.lower() == word_2.lower():
                    #print(word_1, "<->", word_2)
                    return True
            
    for word_2 in text_2:
        if word_2 != None and len(word_2)>2:
            for word_1 in text_1:
                if word_1!= None and word_2.lower() == word_1.lower():
                    #print(word_2, "<->", word_1)
                    return True
    return False

def pick_candidate_tutor(data_frame, title):
    check_multiple = data_frame.copy()
    
    values = ["Dios", "JOSE GREGORIO HERNANDEZ", "Brs"]
    values = map(str.lower, values)
    
    #here we delete dios and JOSE GREGORIO HERNANDEZ and return the first name.
    true_values = check_multiple[check_multiple["contains"]==True]
    t_names = true_values["names"]
    t_names = t_names.to_list()
    #print(check_multiple)
    #print(t_names)
    tutor = check_multiple[~check_multiple['names'].isin(t_names)].copy()
    tutor['tutor_lower'] = tutor['names'].str.lower()
    tutor = tutor[~tutor['tutor_lower'].isin(values)]
    tutor = name_in_title(tutor, title)
    tutor = tutor['names'].to_list()[0]
    return tutor
    

def compare_similarity_t(docs, author, threshold = 0.63):
    aux_docs = docs.copy()
    author = nlp_compare(author)
    for name in aux_docs:
        name_aux = nlp_compare(name)
        if author.similarity(name_aux) >= threshold:
            aux_docs.remove(name)
        #print(name, "<->", author, author.similarity(name_aux))
    return aux_docs

def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

def compare_names(docs, names, threshold = 0.63):
    aux = docs.copy()
    if len(names) > 1:
        for name in names:
            aux = compare_similarity_t(aux, name, threshold)
        return aux[0]    
    else:
        aux = compare_similarity_t(aux, names[0], threshold)
        return aux[0]
    
def get_persons(data):
    first_data = data
    doc = nlp_person(first_data)
    get_persons_list = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            get_persons_list.append(ent.text)
    return get_persons_list

def get_persons_flair(data):
    sentence = Sentence(data)
    tagger.predict(sentence)
    
    list_of_names = []
    for label in sentence.get_labels('ner'):
        # print label value and score
        if label.value == "PER":
            #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
            list_of_names.append(label.data_point.text)
    return(list_of_names)

class Tutor():
    def __init__(self, l):
        self.j = 0
        self.l = l
        
    def compare_all_names(self, file_source, author, threshold = 0.63):
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            #data = clean_accents_lower(data)
            #data[:8000]
            docs = get_persons(data[:8000])
            docs = list(map(clean_accents, docs)) #clean the accents

            authors = split_authors(author)
            tutor = compare_names(docs, authors, threshold)
            self.j+=1
            return tutor
        except:
            self.j+=1
            return "No tutor"
        
    def compare_all_names_flair(self, file_source, author, title):
        self.j+=1
        print(file_source)
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            #data = clean_commas_text(data)
            #data = clean_accents(data)
            docs = data[:8000]
            docs = clean_accents(docs) #clean the accents

            #flair
            sentence = Sentence(docs)
            tagger.predict(sentence)

            list_of_names = []
            for label in sentence.get_labels('ner'):
                # print label value and score
                if label.value == "PER" and len(label.data_point.text)>2:
                    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
                    list_of_names.append(label.data_point.text)
            #print(list_of_names)
            check_multiple = contains_text_list(list_of_names, split_authors(author))
            can_tutor = pick_candidate_tutor(check_multiple, [title])
            #print(can_tutor)
            return can_tutor
        except:
            return "No tutor"
                    
            
            
            
        
        
        
    
    

In [384]:
## names
df_tesis = pd.read_csv('data/thesis_7149_with_resumen_school_complex.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,249,13-Oct-2011,Análisis y evaluación de la aplicación de tecn...,"González P., Héctor V.",2.61 MB,http://saber.ucv.ve/handle/10872/426,http://saber.ucv.ve/bitstream/10872/426/1/TRAB...,NaN,thesis_pdf_all/249.pdf,True,True,False,escuela de ingenieria de petroleo,Escuela de Ingeniería de Petroleo
2,197,30-Sep-2011,Modelo de participación comunitaria del comité...,"Carrillo C., Nancy; Márquez S., Migdalia",280.68 kB,http://saber.ucv.ve/handle/10872/325,http://saber.ucv.ve/bitstream/10872/325/1/MODE...,NaN,thesis_pdf_all/197.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
3,198,30-Sep-2011,Calidad del cuidado brindado por el profesiona...,"Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa",481.18 kB,http://saber.ucv.ve/handle/10872/326,http://saber.ucv.ve/bitstream/10872/326/1/tesi...,NaN,thesis_pdf_all/198.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
4,199,30-Sep-2011,Programa de educación para la prevención del v...,"Colmenares, Suleimy; Contreras, Rosa Y.; Valer...",483.44 kB,http://saber.ucv.ve/handle/10872/327,http://saber.ucv.ve/bitstream/10872/327/1/TESI...,NaN,thesis_pdf_all/199.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7144,7733,30-Jul-2019,Evaluación y selección de sensores en sistemas...,"Sandia I., Alexandra C.",1.4 MB,http://saber.ucv.ve/handle/10872/20088,http://saber.ucv.ve/bitstream/10872/20088/1/TE...,NaN,thesis_pdf_all/7733.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7145,7734,30-Jul-2019,Diseño de una red industrial MODBUS RTU inalám...,"Temprano G., Alejandro J.",1.35 MB,http://saber.ucv.ve/handle/10872/20089,http://saber.ucv.ve/bitstream/10872/20089/1/TE...,NaN,thesis_pdf_all/7734.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7146,7735,31-Jul-2019,Estimación de la demanda y energía eléctrica e...,"Landaeta R., Ana G.",2.45 MB,http://saber.ucv.ve/handle/10872/20090,http://saber.ucv.ve/bitstream/10872/20090/1/La...,NaN,thesis_pdf_all/7735.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7147,7736,31-Jul-2019,Diseño de un sistema de comunicación entre un ...,"Borges F., Gabriel A.",4.76 MB,http://saber.ucv.ve/handle/10872/20091,http://saber.ucv.ve/bitstream/10872/20091/1/TE...,NaN,thesis_pdf_all/7736.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica


In [389]:
df_tesis_100 = df_tesis.copy()
df_tesis_100

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,249,13-Oct-2011,Análisis y evaluación de la aplicación de tecn...,"González P., Héctor V.",2.61 MB,http://saber.ucv.ve/handle/10872/426,http://saber.ucv.ve/bitstream/10872/426/1/TRAB...,NaN,thesis_pdf_all/249.pdf,True,True,False,escuela de ingenieria de petroleo,Escuela de Ingeniería de Petroleo
2,197,30-Sep-2011,Modelo de participación comunitaria del comité...,"Carrillo C., Nancy; Márquez S., Migdalia",280.68 kB,http://saber.ucv.ve/handle/10872/325,http://saber.ucv.ve/bitstream/10872/325/1/MODE...,NaN,thesis_pdf_all/197.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
3,198,30-Sep-2011,Calidad del cuidado brindado por el profesiona...,"Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa",481.18 kB,http://saber.ucv.ve/handle/10872/326,http://saber.ucv.ve/bitstream/10872/326/1/tesi...,NaN,thesis_pdf_all/198.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
4,199,30-Sep-2011,Programa de educación para la prevención del v...,"Colmenares, Suleimy; Contreras, Rosa Y.; Valer...",483.44 kB,http://saber.ucv.ve/handle/10872/327,http://saber.ucv.ve/bitstream/10872/327/1/TESI...,NaN,thesis_pdf_all/199.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7144,7733,30-Jul-2019,Evaluación y selección de sensores en sistemas...,"Sandia I., Alexandra C.",1.4 MB,http://saber.ucv.ve/handle/10872/20088,http://saber.ucv.ve/bitstream/10872/20088/1/TE...,NaN,thesis_pdf_all/7733.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7145,7734,30-Jul-2019,Diseño de una red industrial MODBUS RTU inalám...,"Temprano G., Alejandro J.",1.35 MB,http://saber.ucv.ve/handle/10872/20089,http://saber.ucv.ve/bitstream/10872/20089/1/TE...,NaN,thesis_pdf_all/7734.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7146,7735,31-Jul-2019,Estimación de la demanda y energía eléctrica e...,"Landaeta R., Ana G.",2.45 MB,http://saber.ucv.ve/handle/10872/20090,http://saber.ucv.ve/bitstream/10872/20090/1/La...,NaN,thesis_pdf_all/7735.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7147,7736,31-Jul-2019,Diseño de un sistema de comunicación entre un ...,"Borges F., Gabriel A.",4.76 MB,http://saber.ucv.ve/handle/10872/20091,http://saber.ucv.ve/bitstream/10872/20091/1/TE...,NaN,thesis_pdf_all/7736.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica


In [390]:
l = len(df_tesis_100)
l

7149

In [391]:
tutores = Tutor(l)
l = len(df_tesis_100)
compare_fn = np.vectorize(tutores.compare_all_names_flair, otypes=[np.ndarray])
values_c = compare_fn('../'+ df_tesis_100['path'], df_tesis_100['thesis_author'], df_tesis_100['thesis_title'])
df_tesis_100['thesis_tutor'] = values_c.tolist()

../thesis_pdf_all/1.pdf
../thesis_pdf_all/249.pdf------------------------------------| 0.0% Complete
../thesis_pdf_all/197.pdf------------------------------------| 0.0% Complete
../thesis_pdf_all/198.pdf------------------------------------| 0.0% Complete
../thesis_pdf_all/199.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/200.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/242.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/243.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/244.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/245.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/246.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/247.pdf------------------------------------| 0.2% Complete
../thesis_pdf_all/248.pdf------------------------------------| 0.2% Complete
../thesis_pdf_all/250.pdf---------------------------

../thesis_pdf_all/55.pdf-------------------------------------| 1.5% Complete
../thesis_pdf_all/56.pdf-------------------------------------| 1.5% Complete
../thesis_pdf_all/57.pdf-------------------------------------| 1.5% Complete
../thesis_pdf_all/269.pdf------------------------------------| 1.5% Complete
../thesis_pdf_all/267.pdf------------------------------------| 1.6% Complete
../thesis_pdf_all/1138.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1119.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1108.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1109.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1110.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1111.pdf-----------------------------------| 1.6% Complete
../thesis_pdf_all/1112.pdf-----------------------------------| 1.7% Complete
../thesis_pdf_all/1113.pdf-----------------------------------| 1.7% Complete

../thesis_pdf_all/179.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/180.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/181.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/182.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/183.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/184.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/185.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/186.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/187.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/29.pdf-------------------------------------| 3.1% Complete
../thesis_pdf_all/60.pdf-------------------------------------| 3.1% Complete
../thesis_pdf_all/122.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/710.pdf------------------------------------| 3.1% Complete

../thesis_pdf_all/599.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/328.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/331.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/344.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/332.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/334.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/335.pdf------------------------------------| 4.5% Complete
../thesis_pdf_all/336.pdf------------------------------------| 4.6% Complete
../thesis_pdf_all/337.pdf------------------------------------| 4.6% Complete
../thesis_pdf_all/338.pdf------------------------------------| 4.6% Complete
../thesis_pdf_all/339.pdf------------------------------------| 4.6% Complete
../thesis_pdf_all/327.pdf------------------------------------| 4.6% Complete
../thesis_pdf_all/340.pdf------------------------------------| 4.6% Complete

../thesis_pdf_all/670.pdf------------------------------------| 5.9% Complete
../thesis_pdf_all/671.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/672.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/674.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/676.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/677.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/679.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/680.pdf------------------------------------| 6.0% Complete
../thesis_pdf_all/681.pdf------------------------------------| 6.1% Complete
../thesis_pdf_all/682.pdf------------------------------------| 6.1% Complete
../thesis_pdf_all/683.pdf------------------------------------| 6.1% Complete
../thesis_pdf_all/668.pdf------------------------------------| 6.1% Complete
../thesis_pdf_all/684.pdf------------------------------------| 6.1% Complete

...

../thesis_pdf_all/7903.pdf████████████████████████████████---| 94.1% Complete
../thesis_pdf_all/7874.pdf████████████████████████████████---| 94.1% Complete
../thesis_pdf_all/7872.pdf████████████████████████████████---| 94.1% Complete
../thesis_pdf_all/7878.pdf████████████████████████████████---| 94.1% Complete
../thesis_pdf_all/7876.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/7847.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6556.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6545.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6546.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6547.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6548.pdf████████████████████████████████---| 94.2% Complete
../thesis_pdf_all/6549.pdf████████████████████████████████---| 94.3% Complete
../thesis_pdf_all/6550.pdf████████████████████████████████---| 9

../thesis_pdf_all/6533.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6534.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6522.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6506.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6477.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6478.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6479.pdf████████████████████████████████---| 95.6% Complete
../thesis_pdf_all/6480.pdf████████████████████████████████---| 95.7% Complete
../thesis_pdf_all/6481.pdf████████████████████████████████---| 95.7% Complete
../thesis_pdf_all/6482.pdf████████████████████████████████---| 95.7% Complete
../thesis_pdf_all/6518.pdf████████████████████████████████---| 95.7% Complete
../thesis_pdf_all/6516.pdf████████████████████████████████---| 95.7% Complete
../thesis_pdf_all/6540.pdf████████████████████████████████---| 9

../thesis_pdf_all/8124.pdf█████████████████████████████████--| 97.0% Complete
../thesis_pdf_all/8125.pdf█████████████████████████████████--| 97.0% Complete
../thesis_pdf_all/8126.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8128.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/7806.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8134.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8135.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8123.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8138.pdf█████████████████████████████████--| 97.1% Complete
../thesis_pdf_all/8139.pdf█████████████████████████████████--| 97.2% Complete
../thesis_pdf_all/8140.pdf█████████████████████████████████--| 97.2% Complete
../thesis_pdf_all/8142.pdf█████████████████████████████████--| 97.2% Complete
../thesis_pdf_all/8143.pdf█████████████████████████████████--| 9

../thesis_pdf_all/7842.pdf██████████████████████████████████-| 98.5% Complete
../thesis_pdf_all/7748.pdf██████████████████████████████████-| 98.5% Complete
../thesis_pdf_all/7749.pdf██████████████████████████████████-| 98.5% Complete
../thesis_pdf_all/7750.pdf██████████████████████████████████-| 98.5% Complete
../thesis_pdf_all/7751.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7752.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7753.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7754.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7755.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7756.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7758.pdf██████████████████████████████████-| 98.6% Complete
../thesis_pdf_all/7845.pdf██████████████████████████████████-| 98.7% Complete
../thesis_pdf_all/7759.pdf██████████████████████████████████-| 9

../thesis_pdf_all/7736.pdf██████████████████████████████████-| 100.0% Complete
../thesis_pdf_all/8211.pdf██████████████████████████████████-| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [392]:
df_tesis_100.to_csv("./data/thesis_tutor_7149_v2.csv", index=False)

In [271]:
contains_text('fanny ceballos', 'Programa de educación para la prevención del virus del papiloma humano (vph) dirigido a las usuarias que asisten a la consulta de ginecología del Hospital General del Oeste “Dr. José Gregorio Hernández” de los Magallanes de Catia en el tercer trimestre del año 2005.')

['fanny', 'ceballos']
['Programa', 'de', 'educación', 'para', 'la', 'prevención', 'del', 'virus', 'del', 'papiloma', 'humano', '(vph)', 'dirigido', 'a', 'las', 'usuarias', 'que', 'asisten', 'a', 'la', 'consulta', 'de', 'ginecología', 'del', 'Hospital', 'General', 'del', 'Oeste', '“Dr.', 'José', 'Gregorio', 'Hernández”', 'de', 'los', 'Magallanes', 'de', 'Catia', 'en', 'el', 'tercer', 'trimestre', 'del', 'año', '2005.']


False

In [80]:
df_tesis_100['thesis_tutor']

0    Julio Molina
Name: thesis_tutor, dtype: object

In [11]:
df_tesis_100.to_csv("./data/thesis_tutor_5.csv", index=False)

In [334]:
thesis_number = '243'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data = clean_commas_text(data)
data = clean_accents(data)
data[:8000]

'DESARROLLO DE UN ALGORITMO NUMERICO PARA EL DISENO Y SIMULACION DEL PROCESO DE PRODUCCION DE HIDROCARBUROS MEDIANTE SISTEMAS DE BOMBEO MECANICO TRABAJO ESPECIAL DE GRADO DESARROLLO DE UN ALGORITMO NUMERICO PARA EL DISENO Y SIMULACION DEL PROCESO DE PRODUCCION DE HIDROCARBUROS MEDIANTE SISTEMAS DE BOMBEO MECANICO Presentado Ante la ilustre Universidad Central de Venezuela Por los Brs Gil L Maria Veronica Morillo L Jose R para optar el titulo de Ingeniero Mecanico CARACAS 2003 TRABAJO ESPECIAL DE GRADO DESARROLLO DE UN ALGORITMO NUMERICO PARA EL DISENO Y SIMULACION DEL PROCESO DE PRODUCCION DE HIDROCARBUROS MEDIANTE SISTEMAS DE BOMBEO MECANICO TUTOR ACADEMICO: Prof Ing Jose Luis Perera TUTOR INDUSTRIAL: Prof Ing Julio Gonzalez Presentado Ante la ilustre Universidad Central de Venezuela Por los Brs Gil L Maria Veronica Morillo L Jose R para optar el titulo de Ingeniero Mecanico CARACAS 2003 DEDICATORIA IDEDICATORIA Maria Veronica Gil Lugo: A mis Padres A mis Abuelitos Vicenta y Francisco

In [335]:
author = df_tesis[df_tesis['index'] == int(thesis_number)]
author = author['thesis_author']
author = author.to_list()
author = split_authors(author[0])
author

['Gil L., María V.', ' Morillo L., José R.']

In [336]:
title = df_tesis[df_tesis['index'] == int(thesis_number)]
title = title['thesis_title'].to_list()
title = title
title

['Desarrollo de un algoritmo numérico para el diseño y simulación del proceso de producción de hidrocarburos mediante sistemas de bombeo mecánico']

In [337]:
sentence = Sentence(data[:8000])

In [338]:
tagger.predict(sentence)

In [339]:
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    #print(entity.value)
    print(entity)

The following NER tags are found:
Span[50:54]: "Universidad Central de Venezuela" → ORG (1.0)
Span[56:58]: "Brs Gil" → PER (0.9992)
Span[59:62]: "Maria Veronica Morillo" → PER (1.0)
Span[63:65]: "Jose R" → PER (1.0)
Span[72:73]: "CARACAS" → LOC (1.0)
Span[104:107]: "Jose Luis Perera" → PER (1.0)
Span[112:114]: "Julio Gonzalez" → PER (1.0)
Span[118:122]: "Universidad Central de Venezuela" → ORG (1.0)
Span[124:126]: "Brs Gil" → PER (0.9984)
Span[127:130]: "Maria Veronica Morillo" → PER (1.0)
Span[131:133]: "Jose R" → PER (1.0)
Span[140:141]: "CARACAS" → LOC (1.0)
Span[144:148]: "Maria Veronica Gil Lugo" → PER (0.9999)
Span[155:156]: "Vicenta" → PER (0.9937)
Span[157:158]: "Francisco" → PER (0.7472)
Span[161:163]: "Ana Jenie" → PER (1.0)
Span[164:165]: "Monica" → PER (0.8357)
Span[168:171]: "Lokito " Bello" → PER (0.9992)
Span[176:177]: "Luisa" → PER (0.9927)
Span[178:179]: "Siulyn" → PER (0.6726)
Span[179:183]: "Jose Reinaldo Morillo Linares" → PER (0.9999)
Span[190:192]: "Juan Linares" 

In [340]:
list_of_names = []
for label in sentence.get_labels('ner'):
    # print label value and score
    if label.value == "PER":
        #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
        list_of_names.append(label.data_point.text)
print(list_of_names)
    #print(f'label.value is: "{label.value}"')
    #print(f'label.score is: "{label.score}"')
    # access the data point to which label attaches and print its text
    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')

['Brs Gil', 'Maria Veronica Morillo', 'Jose R', 'Jose Luis Perera', 'Julio Gonzalez', 'Brs Gil', 'Maria Veronica Morillo', 'Jose R', 'Maria Veronica Gil Lugo', 'Vicenta', 'Francisco', 'Ana Jenie', 'Monica', 'Lokito " Bello', 'Luisa', 'Siulyn', 'Jose Reinaldo Morillo Linares', 'Juan Linares', 'Genoveva de Linares', 'Cesar', 'Dios', 'Julio Gonzalez', 'Sharon Escalante', 'Fernando Valencia', 'Marco Pena', 'Carlos Gil', 'Carlos', 'Perera', 'IXGil L Maria Veronica', 'Morillo L Jose']


In [341]:
#compare_similarity(list_of_names, author[1])

In [342]:
#compare_names(list_of_names, author, threshold = 0.43)

In [343]:
check_multiple = contains_text_list(list_of_names, author)
check_multiple

,names,author,author_lower,contains
0,Brs Gil,"Gil L., María V.","gil l., maria v.",False
1,Maria Veronica Morillo,"Gil L., María V.","gil l., maria v.",True
2,Jose R,"Gil L., María V.","gil l., maria v.",False
3,Jose Luis Perera,"Gil L., María V.","gil l., maria v.",False
4,Julio Gonzalez,"Gil L., María V.","gil l., maria v.",False
5,Brs Gil,"Gil L., María V.","gil l., maria v.",False
6,Maria Veronica Morillo,"Gil L., María V.","gil l., maria v.",True
7,Jose R,"Gil L., María V.","gil l., maria v.",False
8,Maria Veronica Gil Lugo,"Gil L., María V.","gil l., maria v.",True
9,Vicenta,"Gil L., María V.","gil l., maria v.",False


In [344]:
check_multiple = contains_text_list(list_of_names, author)
can_tutor = pick_candidate_tutor(check_multiple, title)
can_tutor

'Brs Gil'

In [22]:
can_tutor['names'].to_list()[0]

TypeError: string indices must be integers

In [671]:
compare_names(can_tutor['names'].to_list(), author, threshold = 0.5)

'Aura Perez'

In [663]:
can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
can_tutor

,names,author,author_lower,contains,tutor_lower,is_in_title
6,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
7,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
10,Mylanyela Rosana,"Carrillo C., Nancy","carrillo c., nancy",False,mylanyela rosana,False
11,Omar,"Carrillo C., Nancy","carrillo c., nancy",False,omar,False
13,Venus,"Carrillo C., Nancy","carrillo c., nancy",False,venus,False
16,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
17,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
22,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
23,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
30,Aura Perez,"Márquez S., Migdalia","marquez s., migdalia",False,aura perez,False


In [566]:
contains_text("INGRID PEREZ", title)

False

In [503]:
compare_names(can_tutor['names'].to_list(), author)

'INGRID PEREZ'

In [345]:
author[1].split(" ")

['', 'Contreras,', 'Rosa', 'Y.']

In [346]:
list_of_names[0].split(" ")

['JOSE', 'GREGORIO', 'HERNANDEZ']

In [347]:

contains_text(list_of_names[0], author[0])

False

In [348]:
pd.set_option('display.max_rows', None)
check_multiple = contains_text_list(list_of_names, author)

In [349]:
check_multiple_names = check_multiple['names']
#check_multiple_names.to_list()

In [350]:
true_values = check_multiple[check_multiple["contains"]==True]
t_names = true_values["names"]
t_names.to_list()

['Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy Contreras',
 'Colmenares Suleimy Contreras',
 'Contreras Rosa Yadira',
 'Contreras Rosa Yadira',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira Valera',
 'Valera Carmen Elena',
 'Valera Carmen Elena',
 'Carmen Valera',
 'Valera Carmen Elena',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [361]:
tutor = check_multiple[~check_multiple['names'].isin(t_names.to_list())]
tutor = tutor[lower(tutor["names"]) != "JOSE GREGORIO HERNANDEZ"]
tutor[tutor["names"] != "Dios"]

NameError: name 'lower' is not defined

In [48]:
first_data = data[:8000]
doc = nlp_person(first_data)
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Rosa Yadira Valera',
 'Colmenares Suleimy',
 'Contreras Rosa Yadira',
 'Valera Carmen Elena',
 'Rosa Yadira',
 'Dios',
 'Yadira',
 'Valera Carmen Elena Para',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [49]:
get_persons = list(map(clean_accents_lower, get_persons))

In [57]:
## names
df_tesis = pd.read_csv('data/thesis_7149_with_resumen_school_complex.csv')
authors = df_tesis[df_tesis['index'] == int(thesis_number)]['thesis_author']
authors
authors

4    Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
Name: thesis_author, dtype: object

In [59]:
split_fn = np.vectorize(split_authors)
split_fn(authors).tolist()

ValueError: setting an array element with a sequence

In [465]:
split_authors(df_tesis['thesis_author'][1])

['González P., Héctor V.']

In [466]:
df_tesis['thesis_author']

0                                 De Gregorio S., Luis T.
1                                  González P., Héctor V.
2                Carrillo C., Nancy; Márquez S., Migdalia
3          Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa
4       Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
                              ...                        
7144                              Sandia I., Alexandra C.
7145                            Temprano G., Alejandro J.
7146                                  Landaeta R., Ana G.
7147                                Borges F., Gabriel A.
7148                          Utrera Ávila, Joselin Marly
Name: thesis_author, Length: 7149, dtype: object

In [467]:
tutores = Tutor

In [469]:
compare_fn = np.vectorize(tutores.compare_all_names)
compare_fn()

In [435]:
top10_clean[int(thesis_number)-1]

'Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L'

In [436]:
names = list(map(clean_accents_lower, split_authors(top10_clean[int(thesis_number)-1])))
names 

['Adrian Mogollon Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [437]:
names = split_authors(top10_clean[int(thesis_number)-1])
names

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [438]:
compare_similarity(get_persons, names[0])

Yusmary TUTOR <-> Adrian Mogollón Julia 0.11692980703110051
Angel Moros R. <-> Adrian Mogollón Julia 0.45962000177794815
Echezuria <-> Adrian Mogollón Julia 0.0
Yusmary <-> Adrian Mogollón Julia 0.0
Moros <-> Adrian Mogollón Julia 0.15718233851904745
Angel ivDEDICATORIA <-> Adrian Mogollón Julia 0.37152739068934554
Mil Gracias <-> Adrian Mogollón Julia 0.26608767915043785
Julita <-> Adrian Mogollón Julia 0.0
Yusmy <-> Adrian Mogollón Julia 0.0
Yusmary vAGRADECIMIENTOS <-> Adrian Mogollón Julia 0.0
Carmen Barrios <-> Adrian Mogollón Julia 0.5507273551865355
Profesor Angel Moros <-> Adrian Mogollón Julia 0.41511536453209774
Aida Damas <-> Adrian Mogollón Julia 0.34371654321983364
Angel Moros R. RESUMEN <-> Adrian Mogollón Julia 0.459620014454642


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [439]:
compare_names(get_persons, names, threshold=.46)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [440]:
fn_compare_names = np.vectorize(compare_names)

In [ ]:
values_compare = fn_compare_names

In [431]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [407]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [116]:
compare_similarity_t(get_persons, names[1])

/tmp/ipykernel_5742/1498625836.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


['Angel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Yusmary Valero',
 'Angel Moros R RESUMEN']

In [117]:
compare_similarity(get_persons, names[2])

Angel Moros R <->  Valero Goncalves Yusmary L 0.32039604916347836
Yusmary Valero G <->  Valero Goncalves Yusmary L 0.6697135616296339
Yusmy <->  Valero Goncalves Yusmary L 0.0
Alejandra Gonzalez <->  Valero Goncalves Yusmary L 0.47268456182808843
Angel Moros <->  Valero Goncalves Yusmary L 0.25236357407492055
Yusmary Valero <->  Valero Goncalves Yusmary L 0.7156886585504478
Angel Moros R RESUMEN <->  Valero Goncalves Yusmary L 0.3203960671539872


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [118]:
compare_similarity_t(get_persons, names[2])

['Angel Moros R',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Angel Moros R RESUMEN']

In [40]:
#file_source = '../'+'thesis_pdf_all/1.pdf'

#parsed_pdf = parser.from_file(file_source)
#data = parsed_pdf['content']

#data

In [42]:
#data = data.replace('\n',"")
#data = clean_spaces_text(data)
#data

In [43]:
#first_data = data[:600]
#first_data

In [8]:
doc = nlp_spa("Margarita camina sobre la torta")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Margarita camina 0 16 PERSON


In [9]:
doc = nlp_spa(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Luis T. 288 295 PERSON
2009 352 356 DATE
CONSTANCIA DE APROBACIÓN 360 384 PERSON
13 de mayo de 2009 394 412 DATE
Luis T. De Greg 585 600 PERSON


In [23]:
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_spa = spacy.load('es_core_news_sm')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('es_dep_news_trf')

In [24]:
thesis_number = '3'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data[:8000]

'UNIVERSIDAD CENTRAL DE VENEZUELAUNIVERSIDAD CENTRAL DE VENEZUELA FACULTAD DE HUMANIDADES Y EDUCACIÓN ESCUELA DE BIBLIOTECOLOGÍA Y ARCHIVOLOGÍA LA PRODUCCIÓN DE CONOCIMIENTO: UN ANÁLISIS MÉTRICO EN EL PROGRAMA DE INVESTIGACIÓN Y DESARROLLO DEL IDEC-FAU-UCV Trabajo de licenciatura presentado como requisito de la Escuela de Bibliotecología y Archivología de la Universidad Central de Venezuela para optar al Título de Licenciadas en Bibliotecología y Licenciado en Archivología PRESENTADO: Adrián M Julia Echezuría P Jorge Valero G Yusmary TUTOR: Prof Dr Ángel Moros R Caracas octubre 2008 ADRIÁN Julia Variabilidad de productores y producción de conocimiento del Programa de Investigación y Desarrollo del IDEC-FAU-UCV / Julia Adrián M Jorge Echezuría P Yusmary Valero G ; Tutor Prof Dr Ángel Moros R— Caracas 2008 x 107 h; il ; 30cm Trabajo de Licenciatura (Bibliotecología y Archivología) – Universidad Central de Venezuela 2008 1 ANALISIS MÉTRICO 2 BIBLIOMETRÍA 3 PRODUCCIÓN DE CONOCIMIENTO 4 PID

In [25]:
first_data = data[:8000]
doc = nlp_person(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Adrián M 489 497 PERSON
Julia Echezuría 498 513 PERSON
Jorge Valero G Yusmary TUTOR 516 544 PERSON
Ángel Moros R 554 567 PERSON
Yusmary Valero G 754 770 PERSON
Universidad Central de Venezuela 893 925 ORG
Yusmy 1544 1549 PERSON
Jorge 1764 1769 PERSON
Alejandra González 2140 2158 PERSON
Carmen Barrios 2198 2212 PERSON
Ángel Moros 2266 2277 PERSON
José López 2410 2420 PERSON
Isabel Bello 2664 2676 PERSON
Instituto de Desarrollo Experimental de la Construcción ( 3943 4000 ORG
Programa de Investigación y Desarrollo ( 4013 4053 ORG
1985-2007 5702 5711 DATE
2000-2007 5777 5786 DATE
1985-2007 5864 5873 DATE
2000-2007 5952 5961 DATE
1985-2007 6018 6027 DATE
Julia Adrián Jorge Echezuria 6465 6493 PERSON
Yusmary Valero 6494 6508 PERSON
Ángel Moros R RESUMEN 6516 6537 PERSON
2000-2007 6734 6743 DATE
1985-2006 6817 6826 DATE
2 6967 6968 CARDINAL
3 6999 7000 CARDINAL
two 7513 7516 CARDINAL
2000-2007 7591 7600 DATE
1995-2006 7677 7686 DATE
three 7709 7714 CARDINAL
1 7723 7724 CARDINAL
2 7816 7817 CA

In [27]:
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Adrián M',
 'Julia Echezuría',
 'Jorge Valero G Yusmary TUTOR',
 'Ángel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Jorge',
 'Alejandra González',
 'Carmen Barrios',
 'Ángel Moros',
 'José López',
 'Isabel Bello',
 'Julia Adrián Jorge Echezuria',
 'Yusmary Valero',
 'Ángel Moros R RESUMEN']

In [28]:
## names
df_tesis = pd.read_csv('data/url_thesis_8211_with_pdf_scan.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8168,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8169,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8170,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8171,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [29]:
top10_authors = df_tesis['thesis_author'][:10]
top10_authors = top10_authors.values.tolist()
top10_authors

['De Gregorio S., Luis T.',
 'Rivera, Elizabeth',
 'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.',
 "D'Amico, Walter F.",
 'Cragno, Analia V.; Romero, Marcos A.',
 'Fernández, Fernando J.',
 'Echezuria Perozo, Jorge E.; Rondon, María C.; Gutierrez Luna, Melany',
 'Silva P., Daniel J.',
 'Silva, Carolina del V.',
 'Suárez B., Efren J.']

In [30]:
top10_authors[2]

'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.'

In [31]:
vec_points = np.vectorize(clean_points_text) 

In [32]:
top10_clean = vec_points(top10_authors)
top10_clean = top10_clean.tolist()

In [107]:
def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

In [34]:
compare_similarity(get_persons, str(top10_clean[2]))

Adrián M <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.33937008053670276
Julia Echezuría <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5670012875522249
Jorge Valero G Yusmary TUTOR <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.7070172418471509
Ángel Moros R <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.40224459980277893
Yusmary Valero G <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5975819452444061
Yusmy <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.0
Jorge <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.676725298944286
Alejandra González <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.35765417137227434
Carmen Barrios <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.47139572

/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [35]:
top10_clean[2].split(";")

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']